## Read ReadMe for instructions

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgboost
import re
pd.set_option('display.max_columns', None)

In [62]:
accepted_df = pd.read_csv("datasets/accepted_dataset.csv")
rejected_df = pd.read_csv("datasets/rejected_dataset.csv")

/var/folders/9b/6_8zt65542g2nv99zls1h_nr0000gn/T/ipykernel_25825/3125798192.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv("datasets/accepted_dataset.csv")


In [63]:
accepted_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

In [64]:
rejected_df.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [65]:
np.array(accepted_df.keys())

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'fico_range_low',
       'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'po

In [66]:
np.array(rejected_df.keys())

array(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'], dtype=object)

In [67]:
np.intersect1d(np.array(accepted_df.keys()), np.array(rejected_df.keys()))

array([], dtype=object)

## Formatting

In [68]:
import pandas as pd
import numpy as np
import re 

CORE_FEATURES = [
    'loan_amnt',
    'dti',
    'addr_state',
    'emp_length',
    'purpose',
    'fico_score',       # Standardized FICO/Risk Score
    'application_date', # Standardized date
    'status'            # Target variable (1=Accepted, 0=Rejected)
]

# --- Preprocessing Functions ---

def clean_emp_length(emp_length_str):
    """Cleans the employment length string into numeric years."""
    if pd.isna(emp_length_str) or emp_length_str in ['n/a', 'nan']:
        return np.nan # Use NaN to represent missing/not applicable
    elif emp_length_str == '< 1 year':
        return 0
    elif '10+ years' in emp_length_str:
        return 10
    else:
        # Extract numbers using regex
        match = re.search(r'\d+', emp_length_str)
        if match:
            return int(match.group(0))
        else:
            return np.nan # Cannot parse


# --- Step 1: Add Target Status ---
accepted_df['status'] = 1
rejected_df['status'] = 0

# --- Step 2 & 3: Rename and Select Columns (Rejected) ---
print("Preprocessing Rejected Loans...")
# Rename columns in rejected_df to match accepted_df conventions
rename_map_rejected = {
    'Amount Requested': 'loan_amnt',
    'Application Date': 'application_date_str', # Keep temporary string name
    'Loan Title': 'purpose',
    'Risk_Score': 'fico_score',          # Assuming Risk_Score maps to FICO
    'Debt-To-Income Ratio': 'dti_str',  # Keep temporary string name
    'State': 'addr_state',
    'Employment Length': 'emp_length_str', # Keep temporary string name
    # Add 'Zip Code': 'zip_code' if needed
    # Add 'Policy Code': 'policy_code' if needed
}
# Select only the columns we intend to map plus the status
cols_to_keep_rejected = list(rename_map_rejected.keys()) + ['status']
rejected_df_processed = rejected_df[cols_to_keep_rejected].copy()
rejected_df_processed.rename(columns=rename_map_rejected, inplace=True)
rejected_df_processed = rejected_df_processed.sort_index(axis=1)

print("Preprocessing Accepted Loans...")
# Select corresponding columns plus the status
# We need 'issue_d' for date, 'fico_range_low' for score, raw 'dti', raw 'emp_length'
cols_to_keep_accepted = [
    'loan_amnt',
    'issue_d',          # Use as proxy for application date
    'purpose',
    'dti',              # Raw DTI
    'addr_state',
    'emp_length',       # Raw emp_length
    'fico_range_low',   # Use low end of FICO range
    'status'
    # Add 'zip_code' if needed
    # Add 'policy_code' if needed
]
# Filter out potential columns not present in older datasets if necessary
available_cols_accepted = [col for col in cols_to_keep_accepted if col in accepted_df.columns]
accepted_df_processed = accepted_df[available_cols_accepted].copy()

# Rename accepted columns to standardized names
accepted_df_processed.rename(columns={
    'issue_d': 'application_date_str',
    'fico_range_low': 'fico_score',
    'emp_length': 'emp_length_str', # Keep temp name before cleaning
    'dti': 'dti_str'             # Keep temp name before cleaning
}, inplace=True)

accepted_df_processed = accepted_df_processed.sort_index(axis=1)

print("Standardizing data types...")
accepted_df_processed['application_date_str'] = pd.to_datetime(accepted_df_processed['application_date_str'], format='%b-%Y').dt.strftime('%Y-%m-%d')
accepted_df_processed['application_date_str'] = pd.to_datetime(accepted_df_processed['application_date_str'], errors='coerce')
rejected_df_processed['application_date_str'] = pd.to_datetime(rejected_df_processed['application_date_str'], errors='coerce')
rejected_df_processed['dti_str'] = rejected_df_processed['dti_str'].str.replace('%', '').astype(float)
emp_length_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

accepted_df_processed['emp_length_str'] = accepted_df_processed['emp_length_str'].map(emp_length_mapping)
rejected_df_processed['emp_length_str'] = rejected_df_processed['emp_length_str'].map(emp_length_mapping)
accepted_df_processed['emp_length_str'] = accepted_df_processed['emp_length_str'].astype(float)
rejected_df_processed['emp_length_str'] = rejected_df_processed['emp_length_str'].astype(float)
accepted_df_processed['purpose'] = accepted_df_processed['purpose'].str.replace('_', ' ')
rejected_df_processed['purpose'] = rejected_df_processed['purpose'].str.lower()

accepted_final = accepted_df_processed.copy()
rejected_final = rejected_df_processed.copy()

Preprocessing Rejected Loans...
Preprocessing Accepted Loans...
Standardizing data types...


In [69]:
display(accepted_final.head())
display(rejected_final.head())
display(accepted_final.dtypes)
display(rejected_final.dtypes)

,addr_state,application_date_str,dti_str,emp_length_str,fico_score,loan_amnt,purpose,status
0,PA,2015-12-01,5.91,10.0,675.0,3600.0,debt consolidation,1
1,SD,2015-12-01,16.06,10.0,715.0,24700.0,small business,1
2,IL,2015-12-01,10.78,10.0,695.0,20000.0,home improvement,1
3,NJ,2015-12-01,17.06,10.0,785.0,35000.0,debt consolidation,1
4,PA,2015-12-01,25.37,3.0,695.0,10400.0,major purchase,1


,addr_state,application_date_str,dti_str,emp_length_str,fico_score,loan_amnt,purpose,status
0,NM,2007-05-26,10.00,4.0,693.0,1000.0,wedding covered but no honeymoon,0
1,MA,2007-05-26,10.00,0.0,703.0,1000.0,consolidating debt,0
2,MD,2007-05-27,10.00,1.0,715.0,11000.0,want to consolidate my debt,0
3,MA,2007-05-27,38.64,0.0,698.0,6000.0,waksman,0
4,MD,2007-05-27,9.43,0.0,509.0,1500.0,mdrigo,0


addr_state                      object
application_date_str    datetime64[ns]
dti_str                        float64
emp_length_str                 float64
fico_score                     float64
loan_amnt                      float64
purpose                         object
status                           int64
dtype: object

addr_state                      object
application_date_str    datetime64[ns]
dti_str                        float64
emp_length_str                 float64
fico_score                     float64
loan_amnt                      float64
purpose                         object
status                           int64
dtype: object

In [70]:
# --- Step 6: Combine DataFrames ---
print("Combining datasets...")
combined_df = pd.concat([accepted_final, rejected_final], ignore_index=True)
print(f"Combined dataset shape: {combined_df.shape}")

# # Drop rows where the application date is missing, as it's crucial for temporal split
# combined_df.dropna(subset=['application_date'], inplace=True)
# print(f"Shape after dropping rows with missing application date: {combined_df.shape}")

Combining datasets...
Combined dataset shape: (29909442, 8)


In [71]:
# Drop rows where the application date is missing, as it's crucial for temporal split
combined_df.dropna(subset=['application_date'], inplace=True)
print(f"Shape after dropping rows with missing application date: {combined_df.shape}")

KeyError: ['application_date']

## HAVEN'T DONE MORE BELOW THIS

In [ ]:
# --- Step 6: Combine DataFrames ---
print("Combining datasets...")
combined_df = pd.concat([accepted_final, rejected_final], ignore_index=True)
print(f"Combined dataset shape: {combined_df.shape}")

# Drop rows where the application date is missing, as it's crucial for temporal split
combined_df.dropna(subset=['application_date'], inplace=True)
print(f"Shape after dropping rows with missing application date: {combined_df.shape}")


# --- Step 7: Handle Missing Values ---
print("Handling missing values...")
print("Missing values before imputation:")
print(combined_df.isnull().sum())

# Imputation Strategies (Example - adjust as needed):
# - Median for numerical skewed data (fico, dti, potentially emp_length if treated num)
# - Mean for less skewed numerical (loan_amnt?)
# - Mode or 'Missing' category for categorical (purpose, addr_state)
# - For emp_length, NaN might mean 'not applicable' or truly missing. Median or 0 could work.

impute_median_cols = ['fico_score', 'dti', 'emp_length'] # Example
impute_mean_cols = ['loan_amnt'] # Example

for col in impute_median_cols:
    if col in combined_df.columns:
        median_val = combined_df[col].median()
        combined_df[col].fillna(median_val, inplace=True)
        print(f"Imputed {col} with median: {median_val}")

for col in impute_mean_cols:
     if col in combined_df.columns:
        mean_val = combined_df[col].mean()
        combined_df[col].fillna(mean_val, inplace=True)
        print(f"Imputed {col} with mean: {mean_val:.2f}")

# For categorical, imputation with mode or a new category 'Missing'
# For simplicity here, we'll rely on dropping rows if critical info like FICO was missing earlier
# Or ensure imputation covers the key columns. Check again:
print("\nMissing values after imputation:")
print(combined_df.isnull().sum())
# Consider dropping rows if key features still have NaNs after imputation if appropriate
# combined_df.dropna(subset=['fico_score', 'dti', 'emp_length'], inplace=True)


# --- Step 8: Encode Categorical Features ---
print("Encoding categorical features...")
# Identify categorical columns (object type or explicitly known)
categorical_cols = ['purpose', 'addr_state'] # Add others if kept (e.g., policy_code)

# Use One-Hot Encoding
combined_df_encoded = pd.get_dummies(combined_df, columns=categorical_cols, drop_first=True) # drop_first helps reduce multicollinearity
print(f"Shape after one-hot encoding: {combined_df_encoded.shape}")
print("Example encoded columns:", combined_df_encoded.columns[:10].tolist(), "...") # Show some new columns


# --- Step 9: Extract Year and Temporal Split ---
print("Splitting data by time period...")
combined_df_encoded['year'] = combined_df_encoded['application_date'].dt.year

# Define time periods
recession_years = range(2007, 2013) # 2007-2012 inclusive
post_recession_years = range(2013, 2019) # 2013-2018 inclusive

recession_df = combined_df_encoded[combined_df_encoded['year'].isin(recession_years)].copy()
post_recession_df = combined_df_encoded[combined_df_encoded['year'].isin(post_recession_years)].copy()

# Drop the date and year columns as they are no longer needed for modeling itself
recession_df.drop(columns=['application_date', 'year'], inplace=True)
post_recession_df.drop(columns=['application_date', 'year'], inplace=True)

print(f"Recession Era (2007-2012) dataset shape: {recession_df.shape}")
print(f"Post-Recession Era (2013-2018) dataset shape: {post_recession_df.shape}")

# --- Final Check ---
print("\nPreprocessing Complete.")
print("\nRecession Data Info:")
recession_df.info()
print("\nPost-Recession Data Info:")
post_recession_df.info()

# --- Ready for Modeling ---
# You can now save these processed dataframes or use them directly
# Example:
# recession_df.to_csv('recession_processed.csv', index=False)
# post_recession_df.to_csv('post_recession_processed.csv', index=False)

# Now you have recession_df and post_recession_df ready for your ML models.
# Remember to separate features (X) and target (y = 'status') before training.
# Example for recession data:
# X_recession = recession_df.drop('status', axis=1)
# y_recession = recession_df['status']

Preprocessing Rejected Loans...
Preprocessing Accepted Loans...
Standardizing data types...


/var/folders/9b/6_8zt65542g2nv99zls1h_nr0000gn/T/ipykernel_6951/1503678181.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['application_date'] = pd.to_datetime(df['application_date_str'], errors='coerce')


Accepted features selected: ['loan_amnt', 'dti', 'addr_state', 'emp_length', 'purpose', 'fico_score', 'application_date', 'status']
Rejected features selected: ['loan_amnt', 'dti', 'addr_state', 'emp_length', 'purpose', 'fico_score', 'application_date', 'status']
Combining datasets...
Combined dataset shape: (29909442, 8)
Shape after dropping rows with missing application date: (29909409, 8)
Handling missing values...
Missing values before imputation:
loan_amnt                  0
dti                     1711
addr_state                 0
emp_length           1098262
purpose                    0
fico_score          18497630
application_date           0
status                     0
dtype: int64


/var/folders/9b/6_8zt65542g2nv99zls1h_nr0000gn/T/ipykernel_6951/1503678181.py:176: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df[col].fillna(median_val, inplace=True)


Imputed fico_score with median: 655.0


...

/var/folders/9b/6_8zt65542g2nv99zls1h_nr0000gn/T/ipykernel_6951/1503678181.py:176: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df[col].fillna(median_val, inplace=True)


Imputed emp_length with median: 0.0
Imputed loan_amnt with mean: 13277.88

Missing values after imputation:


/var/folders/9b/6_8zt65542g2nv99zls1h_nr0000gn/T/ipykernel_6951/1503678181.py:182: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df[col].fillna(mean_val, inplace=True)


loan_amnt           0
dti                 0
addr_state          0
emp_length          0
purpose             0
fico_score          0
application_date    0
status              0
dtype: int64
Encoding categorical features...


: 

## Models

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Function to partition datasets by time period
def partition_by_time(accepted_df, rejected_df):
    # For accepted loans
    # Convert issue_d to datetime format
    accepted_df['issue_d'] = pd.to_datetime(accepted_df['issue_d'])

    # Create the time-based partitions
    recession_accepted = accepted_df[(accepted_df['issue_d'] >= '2007-01-01') & 
                                    (accepted_df['issue_d'] <= '2012-12-31')]
    post_recession_accepted = accepted_df[(accepted_df['issue_d'] >= '2013-01-01') & 
                                        (accepted_df['issue_d'] <= '2018-12-31')]

    # For rejected loans
    # Convert Application Date to datetime
    rejected_df['Application Date'] = pd.to_datetime(rejected_df['Application Date'])

    # Create the time-based partitions
    recession_rejected = rejected_df[(rejected_df['Application Date'] >= '2007-01-01') & 
                                    (rejected_df['Application Date'] <= '2012-12-31')]
    post_recession_rejected = rejected_df[(rejected_df['Application Date'] >= '2013-01-01') & 
                                        (rejected_df['Application Date'] <= '2018-12-31')]
    
    return recession_accepted, post_recession_accepted, recession_rejected, post_recession_rejected

def prepare_features(accepted_df, rejected_df):
    # Common features between datasets (with mapping)
    feature_mapping = {
        'accepted': {
            'loan_amnt': 'Amount Requested',
            'zip_code': 'Zip Code',
            'fico_range_low': 'Risk Score',
            'addr_state': 'State',
            'emp_length': 'Employment Length',
            'dti': 'Debt-To-Income Ratio',
            'title': 'Loan Title'
        },
        'rejected': {
            'Amount Requested': 'Amount Requested',
            'Zip Code': 'Zip Code',
            'Risk_Score': 'Risk Score',
            'State': 'State',
            'Employment Length': 'Employment Length',
            'Debt-To-Income Ratio': 'Debt-To-Income Ratio',
            'Loan Title': 'Loan Title'
        }
    }

    # Add target column (1 for accepted, 0 for rejected)
    accepted_features = recession_accepted[list(feature_mapping['accepted'].keys())].copy()
    accepted_features['status'] = 1

    rejected_features = recession_rejected[list(feature_mapping['rejected'].keys())].copy()
    rejected_features['status'] = 0

    accepted_features = accepted_features.rename(columns=feature_mapping['accepted'])
    rejected_features = rejected_features.rename(columns=feature_mapping['rejected'])

    def convert_employment_length(s):
        if pd.isna(s):
            return np.nan
        s = str(s).lower()
        if '<' in s:
            return 0.5  # Treat "< 1 year" as 0 years
        if '+' in s:
            match = re.search(r'(\d+)\+', s)  # Extract number before "+"
            if match:
                return float(match.group(1)) + 2
        # Extract the first number found
        match = re.search(r'\d+', s)
        if match:
            return float(match.group())
        else:
            return np.nan  # Handle cases with no numbers

    # Apply the function to the column
    rejected_features['Employment Length'] = rejected_features['Employment Length'].apply(convert_employment_length)
    accepted_features['Employment Length'] = accepted_features['Employment Length'].apply(convert_employment_length)

    # Remove "%" and convert to float
    rejected_features['Debt-To-Income Ratio'] = (
        rejected_features['Debt-To-Income Ratio']
        .str.replace('%', '')
        .astype(float)
    )

    # Combine datasets
    combined_df = pd.concat([accepted_features, rejected_features], ignore_index=True)
    
    return combined_df

# Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    results = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }
    
    return results

# Function to train and evaluate multiple models
def build_models(X_train, X_test, y_train, y_test):
    # Initialize models with parameter grids for grid search
    models = {
        'RandomForest': {
            'model': RandomForestClassifier(random_state=42),
            'params': {
                'n_estimators': [100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10]
            }
        },
        'KNN': {
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': [3, 5, 7, 10],
                'weights': ['uniform', 'distance']
            }
        },
        'XGBoost': {
            'model': xgb.XGBClassifier(random_state=42),
            'params': {
                'n_estimators': [100, 200],
                'max_depth': [3, 5, 7],
                'learning_rate': [0.01, 0.1, 0.2]
            }
        }
    }
    
    results = {}
    best_models = {}
    
    # Train and evaluate each model
    for model_name, model_info in models.items():
        print(f"Training {model_name}...")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model_info['model'])
        ])
        
        param_grid = {'model__' + key: value for key, value in model_info['params'].items()}
        grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='roc_auc', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        
        best_models[model_name] = grid_search.best_estimator_
        results[model_name] = evaluate_model(grid_search.best_estimator_, X_test, y_test)
        results[model_name]['best_params'] = grid_search.best_params_
        
        print(f"{model_name} trained. Best parameters: {grid_search.best_params_}")
        print(f"ROC AUC: {results[model_name]['roc_auc']:.4f}")
        print("-" * 50)
    
    return results, best_models

# Function to visualize results
def visualize_results(results, era_name):
    # Set up the matplotlib figure
    plt.figure(figsize=(15, 10))
    
    # Bar chart for metrics
    plt.subplot(2, 2, 1)
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    model_names = list(results.keys())
    
    # Create a dataframe for easy plotting
    metrics_df = pd.DataFrame({
        model_name: [results[model_name][metric] for metric in metrics]
        for model_name in model_names
    }, index=metrics)
    
    metrics_df.plot(kind='bar', ax=plt.gca())
    plt.title(f'Model Performance Metrics - {era_name}')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
    
    # Plot confusion matrices
    for i, model_name in enumerate(model_names):
        plt.subplot(2, 2, i+2)
        cm = results[model_name]['confusion_matrix']
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                  xticklabels=['Rejected', 'Accepted'],
                  yticklabels=['Rejected', 'Accepted'])
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.savefig(f'loan_model_comparison_{era_name}.png')
    plt.close()
    
    return f'loan_model_comparison_{era_name}.png'

# Function to visualize feature importance
def visualize_feature_importance(model, feature_names, era_name, model_name):
    # Extract feature importances if available
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_[0])
    else:
        print(f"Model {model_name} doesn't provide feature importances")
        return None
    
    # Create a DataFrame for visualization
    importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    importance_df = importance_df.sort_values('importance', ascending=False).head(15)
    
    # Create visualization
    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=importance_df)
    plt.title(f'Feature Importance - {era_name} - {model_name}')
    plt.tight_layout()
    plt.savefig(f'feature_importance_{era_name}_{model_name}.png')
    plt.close()
    
    return importance_df.head(10)

# Main function to analyze lending patterns
def analyze_lending_patterns(accepted_df, rejected_df):
    # Partition datasets by time period
    partitioned_data = partition_by_time(accepted_df, rejected_df)
    
    era_results = {}
    era_models = {}
    era_feature_importance = {}
    
    # For each era, prepare data, build and evaluate models
    for era_name, era_data in partitioned_data.items():
        print(f"\n{'='*50}\nAnalyzing {era_name} era\n{'='*50}")
        print(f"Accepted loans: {era_data['accepted'].shape[0]}")
        print(f"Rejected loans: {era_data['rejected'].shape[0]}")
        
        # Prepare features for the current era
        combined_df = prepare_features(era_data['accepted'], era_data['rejected'])
        
        # Print information about the dataset
        print(f"Combined dataset shape: {combined_df.shape}")
        print(f"Acceptance rate: {combined_df['status'].mean():.2%}")
        
        # Handle potential class imbalance with sampling if needed
        accepted_count = combined_df[combined_df['status'] == 1].shape[0]
        rejected_count = combined_df[combined_df['status'] == 0].shape[0]
        
        # If severe imbalance, consider downsampling the majority class
        if rejected_count > 3 * accepted_count:
            print(f"Downsampling rejected loans from {rejected_count} to {3 * accepted_count}")
            rejected_sample = combined_df[combined_df['status'] == 0].sample(
                n=min(rejected_count, 3 * accepted_count), 
                random_state=42
            )
            accepted_sample = combined_df[combined_df['status'] == 1]
            combined_df = pd.concat([accepted_sample, rejected_sample])
            print(f"New dataset shape after balancing: {combined_df.shape}")
            print(f"New acceptance rate: {combined_df['status'].mean():.2%}")
        
        # Split features and target
        X = combined_df.drop('status', axis=1)
        y = combined_df['status']
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        print(f"Training set size: {X_train.shape[0]} samples")
        print(f"Test set size: {X_test.shape[0]} samples")
        
        # Build and evaluate models
        results, models = build_models(X_train, X_test, y_train, y_test)
        era_results[era_name] = results
        era_models[era_name] = models
        era_feature_importance[era_name] = {}
        
        # Extract and visualize feature importance for tree-based models
        for model_name, model in models.items():
            if model_name in ['RandomForest', 'XGBoost']:
                # Extract the actual model from the pipeline
                estimator = model.named_steps['model']
                importance_df = visualize_feature_importance(estimator, X.columns, era_name, model_name)
                if importance_df is not None:
                    era_feature_importance[era_name][model_name] = importance_df
        
        # Visualize results
        viz_filename = visualize_results(results, era_name)
        print(f"Results visualization saved to {viz_filename}")
    
    # Compare best models across eras
    best_model_comparison = {}
    for era_name, results in era_results.items():
        # Find the best model for this era based on ROC AUC
        best_model_name = max(results.items(), key=lambda x: x[1]['roc_auc'])[0]
        best_model_metrics = results[best_model_name]
        
        best_model_comparison[era_name] = {
            'best_model': best_model_name,
            'metrics': {
                metric: value for metric, value in best_model_metrics.items() 
                if metric != 'confusion_matrix' and metric != 'best_params'
            }
        }
    
    # Print comparative summary
    print("\n\n" + "="*50)
    print("COMPARATIVE SUMMARY ACROSS ERAS")
    print("="*50)
    
    for era_name, info in best_model_comparison.items():
        print(f"\n{era_name.upper()} ERA:")
        print(f"Best model: {info['best_model']}")
        print("Performance metrics:")
        for metric, value in info['metrics'].items():
            if metric != 'confusion_matrix' and metric != 'best_params':
                print(f"  - {metric}: {value:.4f}")
    
    # Compare feature importance across eras
    print("\n\n" + "="*50)
    print("FEATURE IMPORTANCE COMPARISON ACROSS ERAS")
    print("="*50)
    
    # Find common model type to compare
    common_model_types = set.intersection(
        *[set(era_fi.keys()) for era_fi in era_feature_importance.values()]
    )
    
    if common_model_types:
        model_type = list(common_model_types)[0]
        print(f"\nComparing feature importance for {model_type} across eras:")
        
        # Create a DataFrame to compare top features
        all_features = set()
        for era_name in era_feature_importance:
            if model_type in era_feature_importance[era_name]:
                all_features.update(era_feature_importance[era_name][model_type]['feature'])
        
        comparison_df = pd.DataFrame(0, index=sorted(all_features), columns=era_results.keys())
        
        for era_name in era_feature_importance:
            if model_type in era_feature_importance[era_name]:
                for _, row in era_feature_importance[era_name][model_type].iterrows():
                    feature = row['feature']
                    importance = row['importance']
                    comparison_df.at[feature, era_name] = importance
        
        # Sort by average importance
        comparison_df['avg'] = comparison_df.mean(axis=1)
        comparison_df = comparison_df.sort_values('avg', ascending=False).drop('avg', axis=1)
        
        print("\nTop 10 features by importance across eras:")
        print(comparison_df.head(10))
        
        # Visualize comparison
        plt.figure(figsize=(12, 8))
        comparison_df.head(10).plot(kind='bar')
        plt.title(f'Feature Importance Comparison Across Eras - {model_type}')
        plt.ylabel('Importance Score')
        plt.tight_layout()
        plt.savefig(f'feature_importance_comparison_{model_type}.png')
        plt.close()
        
        print(f"\nFeature importance comparison visualization saved to feature_importance_comparison_{model_type}.png")
    
    return era_results, era_models, best_model_comparison, era_feature_importance

In [63]:
recession_accepted, post_recession_accepted, recession_rejected, post_recession_rejected = partition_by_time(accepted_df, rejected_df)

partitioned_data = {
    "recession": [recession_accepted, recession_rejected],
    "post-recession": [post_recession_accepted, post_recession_rejected]
}
era_results = {}
era_models = {}
era_feature_importance = {}

In [74]:
combined_df = prepare_features(recession_accepted, recession_rejected)
combined_df.head()

print(f"Combined dataset shape: {combined_df.shape}")
print(f"Acceptance rate: {combined_df['status'].mean():.2%}")

# Handle potential class imbalance with sampling if needed
accepted_count = combined_df[combined_df['status'] == 1].shape[0]
rejected_count = combined_df[combined_df['status'] == 0].shape[0]

# If severe imbalance, consider downsampling the majority class
if rejected_count > 3 * accepted_count:
    print(f"Downsampling rejected loans from {rejected_count} to {3 * accepted_count}")
    rejected_sample = combined_df[combined_df['status'] == 0].sample(
        n=min(rejected_count, 3 * accepted_count), 
        random_state=42
    )
    accepted_sample = combined_df[combined_df['status'] == 1]
    combined_df = pd.concat([accepted_sample, rejected_sample])
    print(f"New dataset shape after balancing: {combined_df.shape}")
    print(f"New acceptance rate: {combined_df['status'].mean():.2%}")

Combined dataset shape: (851393, 8)
Acceptance rate: 11.26%
Downsampling rejected loans from 755491 to 287706
New dataset shape after balancing: (383608, 8)
New acceptance rate: 25.00%


In [75]:
X = combined_df.drop('status', axis=1)
y = combined_df['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

Training set size: 306886 samples
Test set size: 76722 samples


In [76]:
# Build and evaluate models
results, models = build_models(X_train, X_test, y_train, y_test)
era_results["recession"] = results
era_models["recession"] = models
era_feature_importance["recession"] = {}

Training RandomForest...


ValueError: 
All the 54 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/cedergrund/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/cedergrund/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cedergrund/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Partition datasets by time period
partitioned_data = partition_by_time(accepted_df, rejected_df)

era_results = {}
era_models = {}
era_feature_importance = {}

# For each era, prepare data, build and evaluate models
for era_name, era_data in partitioned_data.items():
    print(f"\n{'='*50}\nAnalyzing {era_name} era\n{'='*50}")
    print(f"Accepted loans: {era_data['accepted'].shape[0]}")
    print(f"Rejected loans: {era_data['rejected'].shape[0]}")
    
    # Prepare features for the current era
    combined_df = prepare_features(era_data['accepted'], era_data['rejected'])
    
    # Print information about the dataset
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Acceptance rate: {combined_df['status'].mean():.2%}")
    
    # Handle potential class imbalance with sampling if needed
    accepted_count = combined_df[combined_df['status'] == 1].shape[0]
    rejected_count = combined_df[combined_df['status'] == 0].shape[0]
    
    # If severe imbalance, consider downsampling the majority class
    if rejected_count > 3 * accepted_count:
        print(f"Downsampling rejected loans from {rejected_count} to {3 * accepted_count}")
        rejected_sample = combined_df[combined_df['status'] == 0].sample(
            n=min(rejected_count, 3 * accepted_count), 
            random_state=42
        )
        accepted_sample = combined_df[combined_df['status'] == 1]
        combined_df = pd.concat([accepted_sample, rejected_sample])
        print(f"New dataset shape after balancing: {combined_df.shape}")
        print(f"New acceptance rate: {combined_df['status'].mean():.2%}")
    
    # Split features and target
    X = combined_df.drop('status', axis=1)
    y = combined_df['status']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    print(f"Training set size: {X_train.shape[0]} samples")
    print(f"Test set size: {X_test.shape[0]} samples")
    
    # Build and evaluate models
    results, models = build_models(X_train, X_test, y_train, y_test)
    era_results[era_name] = results
    era_models[era_name] = models
    era_feature_importance[era_name] = {}
    
    # Extract and visualize feature importance for tree-based models
    for model_name, model in models.items():
        if model_name in ['RandomForest', 'XGBoost']:
            # Extract the actual model from the pipeline
            estimator = model.named_steps['model']
            importance_df = visualize_feature_importance(estimator, X.columns, era_name, model_name)
            if importance_df is not None:
                era_feature_importance[era_name][model_name] = importance_df
    
    # Visualize results
    viz_filename = visualize_results(results, era_name)
    print(f"Results visualization saved to {viz_filename}")

# Compare best models across eras
best_model_comparison = {}
for era_name, results in era_results.items():
    # Find the best model for this era based on ROC AUC
    best_model_name = max(results.items(), key=lambda x: x[1]['roc_auc'])[0]
    best_model_metrics = results[best_model_name]
    
    best_model_comparison[era_name] = {
        'best_model': best_model_name,
        'metrics': {
            metric: value for metric, value in best_model_metrics.items() 
            if metric != 'confusion_matrix' and metric != 'best_params'
        }
    }

# Print comparative summary
print("\n\n" + "="*50)
print("COMPARATIVE SUMMARY ACROSS ERAS")
print("="*50)

for era_name, info in best_model_comparison.items():
    print(f"\n{era_name.upper()} ERA:")
    print(f"Best model: {info['best_model']}")
    print("Performance metrics:")
    for metric, value in info['metrics'].items():
        if metric != 'confusion_matrix' and metric != 'best_params':
            print(f"  - {metric}: {value:.4f}")

# Compare feature importance across eras
print("\n\n" + "="*50)
print("FEATURE IMPORTANCE COMPARISON ACROSS ERAS")
print("="*50)

# Find common model type to compare
common_model_types = set.intersection(
    *[set(era_fi.keys()) for era_fi in era_feature_importance.values()]
)

if common_model_types:
    model_type = list(common_model_types)[0]
    print(f"\nComparing feature importance for {model_type} across eras:")
    
    # Create a DataFrame to compare top features
    all_features = set()
    for era_name in era_feature_importance:
        if model_type in era_feature_importance[era_name]:
            all_features.update(era_feature_importance[era_name][model_type]['feature'])
    
    comparison_df = pd.DataFrame(0, index=sorted(all_features), columns=era_results.keys())
    
    for era_name in era_feature_importance:
        if model_type in era_feature_importance[era_name]:
            for _, row in era_feature_importance[era_name][model_type].iterrows():
                feature = row['feature']
                importance = row['importance']
                comparison_df.at[feature, era_name] = importance
    
    # Sort by average importance
    comparison_df['avg'] = comparison_df.mean(axis=1)
    comparison_df = comparison_df.sort_values('avg', ascending=False).drop('avg', axis=1)
    
    print("\nTop 10 features by importance across eras:")
    print(comparison_df.head(10))
    
    # Visualize comparison
    plt.figure(figsize=(12, 8))
    comparison_df.head(10).plot(kind='bar')
    plt.title(f'Feature Importance Comparison Across Eras - {model_type}')
    plt.ylabel('Importance Score')
    plt.tight_layout()
    plt.savefig(f'feature_importance_comparison_{model_type}.png')
    plt.close()
    
    print(f"\nFeature importance comparison visualization saved to feature_importance_comparison_{model_type}.png")


In [39]:
results, models, comparison, feature_importance = analyze_lending_patterns(accepted_df, rejected_df)


Analyzing recession era
Accepted loans: 95902
Rejected loans: 755491


: 